This notebook transforms the dataframe from wide to long format.

- Takes as an input a df with columns for sentiment, rating and number of reviews both before and after (6 columns)
- It output a df in long format with 3 columns: rating, sentiment and number of reviews. For each initial row there are now 2 rows, one referring to the value contained in the inital before columns and the other contained in the original after columns. We know that we are referring to the before or after observation by looking at the "After" column

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("all_data_FINAL_FILT.csv")
df.head(2)

,Unnamed: 0,year,category,nominee,workers,winner,grammy_date,workers_clean,nominee_semiclean,url_final,rating_before,rating_after,sentiment_before,sentiment_after,num_reviews_before,num_reviews_after
0,34,2002,Best R&B Album,Songs In A Minor,Alicia Keys (artist/producer),True,2002-02-27,alicia-keys,songs-in-a-minor,https://rateyourmusic.com/release/album/alicia...,5.0,3.725000,0.998884,0.803954,2.0,41.0
1,38,2002,Best Alternative Music Album,Parachutes,"Guy Berryman (producer), Jon Buckland (produce...",True,2002-02-27,coldplay,parachutes,https://rateyourmusic.com/release/album/coldpl...,5.0,3.632353,0.998897,0.808332,3.0,208.0


In [3]:
# Dropping unnecessary index
df = df.drop(columns='Unnamed: 0')

# Creating 2 copies of the df
df_before = df.copy()
df_before = df_before.drop(columns=["rating_after","sentiment_after", "num_reviews_after"])
df_after = df.copy()
df_after = df_after.drop(columns=["rating_before","sentiment_before", "num_reviews_before"])

# Each df contains the information about [rating, sentiment and num_reviews] relative to its category [before, after] 
df_before = df_before.rename(columns={"rating_before":"rating","sentiment_before":"sentiment", "num_reviews_before":"num_reviews"})
df_after = df_after.rename(columns={"rating_after":"rating","sentiment_after":"sentiment", "num_reviews_after":"num_reviews"})

# Column identifying which observation it refers to
df_before['After'] = False
df_after['After'] = True

# We concatenate the 2 intermediaries df
all_df = pd.concat([df_before, df_after])
all_df.shape
all_df = all_df.sort_values(by=["url_final","After"]).reset_index(drop=True)

In [4]:
list_mainstream = ["Album of the Year","Best Alternative Music Album", "Best Contemporary R&B Album", "Best Country Album", "Best Electronic/Dance Album", "Best Pop Vocal Album","Best Rap Album", "Best R&B Album"]

all_df["mainstream"] = [1 if all_df.category[x] in list_mainstream else 0 for x in range(len(all_df))]

In [5]:
all_df.head(4)

,year,category,nominee,workers,winner,grammy_date,workers_clean,nominee_semiclean,url_final,rating,sentiment,num_reviews,After,mainstream
0,2018,Best Folk Album,Mental Illness,Aimee Mann (artist),True,2018-01-28,aimee-mann,mental-illness,https://rateyourmusic.com/release/album/aimee-...,3.785714,0.855841,7.0,False,0
1,2018,Best Folk Album,Mental Illness,Aimee Mann (artist),True,2018-01-28,aimee-mann,mental-illness,https://rateyourmusic.com/release/album/aimee-...,3.500000,0.998836,2.0,True,0
2,2016,Best Alternative Music Album,Sound & Color.,Alabama Shakes,True,2016-02-15,alabama-shakes,sound-and-color,https://rateyourmusic.com/release/album/alabam...,3.142857,0.732128,15.0,False,1
3,2016,Best Alternative Music Album,Sound & Color.,Alabama Shakes,True,2016-02-15,alabama-shakes,sound-and-color,https://rateyourmusic.com/release/album/alabam...,3.363636,0.725554,11.0,True,1


In [6]:
all_df.to_csv("all_data_FINAL_FILT_LONG.csv")